# AbsPL (Absorption and Photoluminescence) Analysis

## How to use this notebook:
1. Select batches to analyze (only batches of type "hysprint_batch" are considered)
2. The data will be loaded into a pandas DataFrame
3. Use the plotting tools to visualize your data:
   - Create scatter plots for comparing two parameters
   - Use box plots to analyze parameter distributions
4. Access advanced features for data table viewing and statistics

In [7]:
%matplotlib ipympl
%load_ext autoreload
%autoreload 2
import os
import base64
import io
import sys
import ipywidgets as widgets
import plotly.graph_objects as go
import plotly.express as px
from IPython.display import display, Markdown, HTML
import pandas as pd
import numpy as np
import json

sys.path.append(os.path.dirname(os.getcwd()))
from api_calls import get_ids_in_batch, get_sample_description, get_all_eqe as get_all_abspl
import batch_selection
import plotting_utils
import access_token

url_base ="https://nomad-hzb-se.de"
url = f"{url_base}/nomad-oasis/api/v1"
token = access_token.get_token(url)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
warning_sign = "\u26A0"

out = widgets.Output()
out2 = widgets.Output()
read = widgets.Output()
dynamic_content = widgets.Output()  # For dynamically updated content
results_content = widgets.Output(layout={
    # 'border': '1px solid black',  # Optional: adds a border to the widget
    'max-height': '1000px',  # Set the height
    'overflow': 'scroll',  # Adds a scrollbar if content overflows
    })
cell_edit = widgets.VBox() 

default_variables = widgets.Dropdown(
    options=['sample name', 'batch',"sample description", 'custom'],
    index=0,
    description='name preset:',
    disabled=False,
    tooltip="Presets for how the samples will be named in the plot"
)
data = None


#this function takes sample ids and returns the eqe curves and parameters as Dataframes
def get_abspl_data(try_sample_ids, variation):
    #parameters of single eqe measurement
    abs_pl_params_names = ["luminescence_quantum_yield","quasi_fermi_level_splitting","i_voc","bandgap","derived_jsc"]
    #make api call, result has everything in json format
    all_abspl = get_all_abspl(url, token, try_sample_ids, eqe_type="HySprint_AbsPLMeasurement")

    existing_sample_ids = pd.Series(all_abspl.keys())

    # Check if there's any EQE data
    if len(existing_sample_ids) == 0:
        return None  # Return None value to indicate no data

    sample_params_list = []
    for sample_id, sample_data in all_abspl.items():
        for abspl_entry in sample_data:
            df = pd.DataFrame(abspl_entry[0]["results"], columns=abs_pl_params_names)
            df["sample_id"] = sample_id
            df["variation"] = variation.get(sample_id, '')
            df["name"] = abspl_entry[0].get("name", '')
            sample_params_list.append(df)

      
    # Only try to concatenate if there's data
    if sample_params_list:
        return pd.concat(sample_params_list)
    return None

def create_plotting_widgets():
    """Create widgets for plotting functionality"""
    global data
    
    if data is None:
        return widgets.HTML("No data available for plotting.")
    
    # Get available columns for plotting
    numeric_cols = data.select_dtypes(include=['float64', 'int64']).columns.tolist()
    category_cols = ['sample_id', 'variation', 'name']
    all_cols = numeric_cols + category_cols
    
    # Create dropdowns for X and Y axes
    x_dropdown = widgets.Dropdown(
        options=all_cols,
        value=numeric_cols[0] if numeric_cols else all_cols[0],
        description='X-axis:',
        tooltip="Select column for X-axis"
    )
    
    y_dropdown = widgets.Dropdown(
        options=numeric_cols,
        value=numeric_cols[1] if len(numeric_cols) > 1 else numeric_cols[0],
        description='Y-axis:',
        tooltip="Select column for Y-axis"
    )
    
    # Create dropdown for color column
    color_dropdown = widgets.Dropdown(
        options=['None'] + category_cols,
        value='variation',
        description='Color by:',
        tooltip="Select column to color points by"
    )
    
    # Create dropdown for plot type
    plot_type_dropdown = widgets.Dropdown(
        options=['Scatter plot', 'Box plot'],
        value='Scatter plot',
        description='Plot type:',
        tooltip="Select type of plot to display"
    )
    
    # Create button to generate the plot
    plot_button = widgets.Button(
        description='Generate Plot',
        button_style='success',
        tooltip='Click to generate the plot'
    )
    
    # Create plot output area
    plot_output = widgets.Output()
    
    # Define the plotting function
    def on_plot_button_clicked(b):
        with plot_output:
            plot_output.clear_output(wait=True)
            x_col = x_dropdown.value
            y_col = y_dropdown.value
            color_col = None if color_dropdown.value == 'None' else color_dropdown.value
            plot_type = plot_type_dropdown.value
            
            if plot_type == 'Scatter plot':
                fig = px.scatter(
                    data, 
                    x=x_col, 
                    y=y_col, 
                    color=color_col,
                    hover_data=['name', 'sample_id'],
                    title=f'Scatter Plot: {y_col} vs {x_col}'
                )
                fig.update_layout(height=600, width=800)
                display(fig)
                
            elif plot_type == 'Box plot':
                if color_col and color_col != 'None':
                    fig = px.box(
                        data, 
                        x=color_col, 
                        y=y_col, 
                        title=f'Box Plot of {y_col} by {color_col}'
                    )
                    fig.update_layout(height=600, width=800)
                    display(fig)
                else:
                    fig = px.box(
                        data, 
                        y=y_col, 
                        title=f'Box Plot of {y_col}'
                    )
                    fig.update_layout(height=600, width=800)
                    display(fig)
    
    # Connect the button to the function
    plot_button.on_click(on_plot_button_clicked)
    
    # Layout the widgets
    controls = widgets.VBox([
        widgets.HBox([x_dropdown, y_dropdown]),
        widgets.HBox([color_dropdown, plot_type_dropdown]),
        plot_button
    ])
    
    return widgets.VBox([controls, plot_output])

def on_load_data_clicked(batch_ids_selector):
    #global dictionary to hold data
    global data
    dynamic_content.clear_output()
    with out:
        out.clear_output()
        print("Loading Data")

        try_sample_ids = get_ids_in_batch(url, token, batch_ids_selector.value)

        #extract EQE here
        identifiers = get_sample_description(url, token, list(try_sample_ids))
        data = get_abspl_data(try_sample_ids, identifiers)

        # Check if EQE data was found
        if data is None:
            out.clear_output()
            print("The batches selected don't contain any AbsPL measurements")
            return

        out.clear_output()
        print("Data Loaded")
        
        # Create and display plotting widgets once data is loaded
        with dynamic_content:
            dynamic_content.clear_output(wait=True)
            # Display data summary
            display(widgets.HTML("<h3>Data Summary</h3>"))
            display(data.describe())
            
            # Display plotting widgets
            display(widgets.HTML("<h3>Plotting Tools</h3>"))
            plotting_widgets = create_plotting_widgets()
            display(plotting_widgets)



display(plotting_utils.create_manual("eqe_manual.md"))

display(batch_selection.create_batch_selection(url, token, on_load_data_clicked))
display(out)
display(dynamic_content)  # This will be updated dynamically with the variables menu

Output()

Output()

In [11]:
# Additional functions for data visualization and export

def create_data_table_view():
    """Create a data table view with filtering capabilities"""
    global data
    
    if data is None:
        return widgets.HTML("No data available for displaying.")
    
    # Create output for the table display
    table_output = widgets.Output()
    
    # Create column selection for table view
    columns = data.columns.tolist()
    column_selector = widgets.SelectMultiple(
        options=columns,
        value=columns[:5],  # Default to first 5 columns
        description='Columns:',
        disabled=False,
        layout=widgets.Layout(width='50%', height='100px')
    )
    
    # Add button to update table view
    update_button = widgets.Button(
        description='Update Table',
        button_style='info',
        tooltip='Click to update the table view'
    )
    
    # Add export button
    export_button = widgets.Button(
        description='Export to CSV',
        button_style='warning',
        tooltip='Click to export current data to CSV'
    )
    
    # Function to update table view
    def update_table_view(b):
        with table_output:
            table_output.clear_output(wait=True)
            if column_selector.value:
                display(data[list(column_selector.value)].head(20))
                display(widgets.HTML(f"<p>Showing top 20 rows of {len(data)} total rows</p>"))
            else:
                display(widgets.HTML("<p>Please select at least one column to display</p>"))
    
    # Function to export data to CSV
    def trigger_download(text, filename, kind='text/json'):
        content_b64 = base64.b64encode(text.encode()).decode()
        data_url = f'data:{kind};charset=utf-8;base64,{content_b64}'
        js_code = f"""
            var a = document.createElement('a');
            a.setAttribute('download', '{filename}');
            a.setAttribute('href', '{data_url}');
            a.click()
        """
        with download_content:
            download_content.clear_output()
            display(HTML(f'<script>{js_code}</script>'))

    def export_to_csv(e=None):
        abspl_data = io.StringIO()
        data.to_csv(abspl_data)
        timestamp = pd.Timestamp.now().strftime("%Y%m%d_%H%M%S")
        filename = f"abspl_data_export_{timestamp}.csv"
        trigger_download(abspl_data.getvalue(), filename, kind='text/plain')
        with table_output:
            table_output.clear_output(wait=True)
            display(widgets.HTML(f"<p>Data exported to {filename}</p>"))
            update_table_view(None)
    
    # Connect buttons to functions
    update_button.on_click(update_table_view)
    export_button.on_click(export_to_csv)
    
    # Layout the widgets
    controls = widgets.VBox([
        widgets.HTML("<h4>Select columns to display:</h4>"),
        column_selector,
        widgets.HBox([update_button, export_button])
    ])
    
    # Initialize table view
    with table_output:
        if column_selector.value:
            display(data[list(column_selector.value)].head(20))
            display(widgets.HTML(f"<p>Showing top 20 rows of {len(data)} total rows</p>"))
    
    return widgets.VBox([controls, table_output])

def create_statistics_view():
    """Create a view for statistical analysis of the data"""
    global data
    
    if data is None:
        return widgets.HTML("No data available for statistics.")
    
    # Create output for statistics display
    stats_output = widgets.Output()
    
    # Get numeric columns for statistics
    numeric_cols = data.select_dtypes(include=['float64', 'int64']).columns.tolist()
    
    # Create column selection for statistics
    column_selector = widgets.Dropdown(
        options=numeric_cols,
        value=numeric_cols[0] if numeric_cols else None,
        description='Column:',
        disabled=False
    )
    
    # Create groupby column selection
    category_cols = ['sample_id', 'variation', 'name']
    groupby_selector = widgets.Dropdown(
        options=['None'] + category_cols,
        value='variation',
        description='Group by:',
        disabled=False
    )
    
    # Add button to calculate statistics
    stats_button = widgets.Button(
        description='Calculate Stats',
        button_style='info',
        tooltip='Click to calculate statistics'
    )
    
    # Function to calculate and display statistics
    def calculate_statistics(b):
        with stats_output:
            stats_output.clear_output(wait=True)
            col = column_selector.value
            groupby = groupby_selector.value
            
            if col:
                # Overall statistics
                display(widgets.HTML(f"<h4>Overall Statistics for {col}</h4>"))
                stats = data[col].describe()
                display(stats)
                
                # Grouped statistics if groupby is selected
                if groupby != 'None':
                    display(widgets.HTML(f"<h4>Statistics for {col} grouped by {groupby}</h4>"))
                    grouped_stats = data.groupby(groupby)[col].describe()
                    display(grouped_stats)
                    
                    # Create a comparison boxplot
                    fig = px.box(data, x=groupby, y=col, title=f"Comparison of {col} by {groupby}")
                    display(fig)
            else:
                display(widgets.HTML("<p>Please select a column for statistics</p>"))
    
    # Connect button to function
    stats_button.on_click(calculate_statistics)
    
    # Layout the widgets
    controls = widgets.VBox([
        widgets.HBox([column_selector, groupby_selector]),
        stats_button
    ])
    
    return widgets.VBox([controls, stats_output])

# Create tabs for different functionalities
def display_advanced_features():
    global data
    
    if data is None:
        return widgets.HTML("<h3>Please load data first</h3>")
    
    # Create tabs
    tab1 = create_data_table_view()
    tab2 = create_statistics_view()
    
    tabs = widgets.Tab()
    tabs.children = [tab1, tab2]
    tabs.titles = ['Data Table', 'Statistics']
    tabs.set_title(0, 'Data Table')
    tabs.set_title(1, 'Statistics')
    
    return tabs

# Button to show/hide advanced features
advanced_button = widgets.Button(
    description='Toggle Advanced Features',
    button_style='primary',
    tooltip='Click to show/hide advanced data features'
)

advanced_output = widgets.Output()

def on_advanced_button_clicked(b):
    with advanced_output:
        advanced_output.clear_output(wait=True)
        display(display_advanced_features())

advanced_button.on_click(on_advanced_button_clicked)

display(widgets.HTML("<h2>Advanced Data Analysis</h2>"))
display(advanced_button)
display(advanced_output)
download_content = widgets.Output()
download_area = widgets.VBox([download_content])
display(download_area)

HTML(value='<h2>Advanced Data Analysis</h2>')

Button(button_style='primary', description='Toggle Advanced Features', style=ButtonStyle(), tooltip='Click to …

Output()